<a href="https://colab.research.google.com/github/anandt555/BigData-Practice/blob/main/squared_error_cifar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
import torch.nn.functional as F

# Download CIFAR-10 dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_dataset = torchvision.datasets.CIFAR10(root='./data1', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data1', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

# LeNet model with Dirichlet distribution for Evidential Deep Learning and mixup
class LeNetDirichletCIFARMixup(nn.Module):
    def __init__(self):
        super(LeNetDirichletCIFARMixup, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(8*8*64, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool1(torch.relu(self.conv1(x)))
        x = self.pool2(torch.relu(self.conv2(x)))
        x = x.view(-1, 8*8*64)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)

        alpha = torch.abs(x) + 1

        u = 10 / torch.sum(alpha, dim=1, keepdim=True)

        prob = alpha / torch.sum(alpha, 1, keepdim=True)

        return prob, u, alpha

# Updated mixup function for Evidential Deep Learning
def mixup_data(x, y, alpha=0.1):
    lam = np.random.beta(alpha, alpha)
    batch_size = x.size()[0]
    index = torch.randperm(batch_size)
    mixed_x = lam * x + (1 - lam) * x[index, :]

    # Convert labels to one-hot encoding
    y_onehot = torch.zeros(y.size(0), 10).to(x.device)
    y_onehot.scatter_(1, y.view(-1, 1).long(), 1)
    mixed_y = lam * y_onehot + (1 - lam) * y_onehot[index, :]
    mixed_y = mixed_y.argmax(dim=1)
    return mixed_x, mixed_y


# Define custom loss function and KL divergence
def KL(alpha, num_classes=10):
    one = torch.ones((1, num_classes), dtype=torch.float32)
    S = torch.sum(alpha, dim=1, keepdim=True)

    kl = torch.lgamma(S) - torch.sum(torch.lgamma(alpha), dim=1, keepdim=True) + \
         torch.sum(torch.lgamma(one), dim=1, keepdim=True) - torch.lgamma(torch.sum(one, dim=1, keepdim=True)) + \
         torch.sum((alpha - one) * (torch.digamma(alpha) - torch.digamma(S)), dim=1, keepdim=True)

    return kl

def custom_loss(y_true, output):
    epochs = [1]

    y_evidence = torch.relu(output)
    alpha = y_evidence + 1
    S = torch.sum(alpha, dim=1, keepdim=True)
    p = alpha / S

    err = torch.sum(torch.pow((y_true - p), 2), dim=1, keepdim=True)
    var = torch.sum(alpha * (S - alpha) / (S * S * (S + 1)), dim=1, keepdim=True)

    l = torch.sum(err + var, dim=1, keepdim=True)

    kl = torch.min(torch.tensor(1.0), torch.tensor(epochs[0] / 50)) * torch.sum(KL((1 - y_true) * (alpha) + y_true))
    return torch.sum(l + kl)

# Train LeNet model with Dirichlet distribution and mixup for Evidential Deep Learning
lenet_dirichlet_CIFAR_mixup = LeNetDirichletCIFARMixup()
optimizer_dirichlet = optim.Adam(lenet_dirichlet_CIFAR_mixup.parameters(), lr=0.001)

for epoch in range(10):
    lenet_dirichlet_CIFAR_mixup.train()
    for inputs, labels in train_loader:
        inputs, labels = mixup_data(inputs, labels.unsqueeze(1))  # Assuming labels are one-dimensional

        optimizer_dirichlet.zero_grad()
        outputs, _, _ = lenet_dirichlet_CIFAR_mixup(inputs)

        # Cast labels back to Long data type
        labels = labels.squeeze().long()

        loss = custom_loss(F.one_hot(labels, num_classes=10).float(), outputs)  # Using custom loss
        loss.backward()
        optimizer_dirichlet.step()

    lenet_dirichlet_CIFAR_mixup.eval()
    correct_train = 0
    total_train = 0
    with torch.no_grad():
        for inputs, labels in train_loader:
            outputs, uncertainty_train , _ = lenet_dirichlet_CIFAR_mixup(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs, uncertainty_test , _ = lenet_dirichlet_CIFAR_mixup(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()

    print('epoch %d - training accuracy: %2.4f \t training uncertainty: %2.4f \t testing accuracy: %2.4f \t testing uncertainty: %2.4f' %
          (epoch+1, correct_train / total_train, uncertainty_train.mean(), correct_test / total_test, uncertainty_test.mean()))


100%|██████████| 170498071/170498071 [00:13<00:00, 12501750.67it/s]


Extracting ./data1/cifar-10-python.tar.gz to ./data1
Files already downloaded and verified
epoch 1 - training accuracy: 0.5522 	 training uncertainty: 0.5751 	 testing accuracy: 0.5426 	 testing uncertainty: 0.4966
epoch 2 - training accuracy: 0.6368 	 training uncertainty: 0.5114 	 testing accuracy: 0.6120 	 testing uncertainty: 0.3713
epoch 3 - training accuracy: 0.6838 	 training uncertainty: 0.5464 	 testing accuracy: 0.6391 	 testing uncertainty: 0.3731
epoch 4 - training accuracy: 0.7174 	 training uncertainty: 0.4857 	 testing accuracy: 0.6649 	 testing uncertainty: 0.2834
epoch 5 - training accuracy: 0.7330 	 training uncertainty: 0.3691 	 testing accuracy: 0.6651 	 testing uncertainty: 0.2611
epoch 6 - training accuracy: 0.7658 	 training uncertainty: 0.5296 	 testing accuracy: 0.6879 	 testing uncertainty: 0.2834
epoch 7 - training accuracy: 0.7723 	 training uncertainty: 0.2332 	 testing accuracy: 0.6836 	 testing uncertainty: 0.2268
epoch 8 - training accuracy: 0.7961 	 tra

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
import torch.nn.functional as F

# Download CIFAR-10 dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_dataset = torchvision.datasets.CIFAR10(root='./data1', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data1', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

# LeNet model with Dirichlet distribution for Evidential Deep Learning
class LeNetDirichletCIFAR(nn.Module):
    def __init__(self):
        super(LeNetDirichletCIFAR, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2)  # Adjust input channels
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(8*8*64, 512)  # Adjust the input size for fully connected layer
        self.fc2 = nn.Linear(512, 10)  # Adjust output classes

    def forward(self, x):
        x = self.pool1(torch.relu(self.conv1(x)))
        x = self.pool2(torch.relu(self.conv2(x)))
        x = x.view(-1, 8*8*64)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)

        alpha = torch.abs(x) + 1  # Use absolute values for simplicity, adjust as needed

        u = 10 / torch.sum(alpha, dim=1, keepdim=True)

        prob = alpha / torch.sum(alpha, 1, keepdim=True)

        return prob, u, alpha

# Define custom loss function and KL divergence
def KL(alpha, num_classes=10):
    one = torch.ones((1, num_classes), dtype=torch.float32)
    S = torch.sum(alpha, dim=1, keepdim=True)

    kl = torch.lgamma(S) - torch.sum(torch.lgamma(alpha), dim=1, keepdim=True) + \
         torch.sum(torch.lgamma(one), dim=1, keepdim=True) - torch.lgamma(torch.sum(one, dim=1, keepdim=True)) + \
         torch.sum((alpha - one) * (torch.digamma(alpha) - torch.digamma(S)), dim=1, keepdim=True)

    return kl

def custom_loss(y_true, output):
    epochs = [1]

    y_evidence = torch.relu(output)
    alpha = y_evidence + 1
    S = torch.sum(alpha, dim=1, keepdim=True)
    p = alpha / S

    err = torch.sum(torch.pow((y_true - p), 2), dim=1, keepdim=True)
    var = torch.sum(alpha * (S - alpha) / (S * S * (S + 1)), dim=1, keepdim=True)

    l = torch.sum(err + var, dim=1, keepdim=True)

    kl = torch.min(torch.tensor(1.0), torch.tensor(epochs[0] / 50)) * torch.sum(KL((1 - y_true) * (alpha) + y_true))
    return torch.sum(l + kl)

# Train LeNet model with Dirichlet distribution for Evidential Deep Learning
lenet_dirichlet_CIFAR = LeNetDirichletCIFAR()
optimizer_dirichlet = optim.Adam(lenet_dirichlet_CIFAR.parameters(), lr=0.001)

for epoch in range(10):
    lenet_dirichlet_CIFAR.train()
    for inputs, labels in train_loader:
        optimizer_dirichlet.zero_grad()
        outputs, _, _ = lenet_dirichlet_CIFAR(inputs)
        loss = custom_loss(F.one_hot(labels, num_classes=10).float(), outputs)  # Using custom loss
        loss.backward()
        optimizer_dirichlet.step()

    lenet_dirichlet_CIFAR.eval()
    correct_train = 0
    total_train = 0
    with torch.no_grad():
        for inputs, labels in train_loader:
            outputs, uncertainty_train, _ = lenet_dirichlet_CIFAR(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs, uncertainty_test, _ = lenet_dirichlet_CIFAR(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()

    print('epoch %d - training accuracy: %2.4f \t training uncertainty: %2.4f \t testing accuracy: %2.4f \t testing uncertainty: %2.4f' %
          (epoch+1, correct_train / total_train, uncertainty_train.mean(), correct_test / total_test, uncertainty_test.mean()))

Files already downloaded and verified
Files already downloaded and verified
epoch 1 - training accuracy: 0.5336 	 training uncertainty: 0.4733 	 testing accuracy: 0.5274 	 testing uncertainty: 0.4389
epoch 2 - training accuracy: 0.6552 	 training uncertainty: 0.4384 	 testing accuracy: 0.6229 	 testing uncertainty: 0.3538
epoch 3 - training accuracy: 0.7160 	 training uncertainty: 0.3909 	 testing accuracy: 0.6621 	 testing uncertainty: 0.3833
epoch 4 - training accuracy: 0.7549 	 training uncertainty: 0.3013 	 testing accuracy: 0.6859 	 testing uncertainty: 0.2759
epoch 5 - training accuracy: 0.7840 	 training uncertainty: 0.3863 	 testing accuracy: 0.6945 	 testing uncertainty: 0.2981
epoch 6 - training accuracy: 0.8001 	 training uncertainty: 0.2216 	 testing accuracy: 0.6983 	 testing uncertainty: 0.2479
epoch 7 - training accuracy: 0.8217 	 training uncertainty: 0.3593 	 testing accuracy: 0.7005 	 testing uncertainty: 0.2493
epoch 8 - training accuracy: 0.8373 	 training uncertain

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
import torch.nn.functional as F

# Download CIFAR-10 dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_dataset = torchvision.datasets.CIFAR10(root='./data1', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data1', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

# LeNet model with Dirichlet distribution for Evidential Deep Learning and regmixup
class LeNetDirichletCIFARMixup(nn.Module):
    def __init__(self):
        super(LeNetDirichletCIFARMixup, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(8*8*64, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool1(torch.relu(self.conv1(x)))
        x = self.pool2(torch.relu(self.conv2(x)))
        x = x.view(-1, 8*8*64)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)

        alpha = torch.abs(x) + 1

        u = 10 / torch.sum(alpha, dim=1, keepdim=True)

        prob = alpha / torch.sum(alpha, 1, keepdim=True)

        return prob, u, alpha

# Updated regmixup function for Evidential Deep Learning
def regmixup(x, y, beta=1.0, cutmix_min=0.0, cutmix_max=0.8):
    lambda_ = np.random.beta(beta, beta)
    r = np.random.rand(1)
    if r < cutmix_min:
        return x, y
    elif r > cutmix_max:
        return x.clone(), y.clone()

    # CutMix regularization (optional)
    indices = torch.randperm(x.shape[0])  # Shuffle for random mask generation
    mask = torch.ones(x.size()).float()
    bbx_y = np.random.randint(0, x.size(-2) - int(cutmix_max * x.size(-2)))
    bbx_x = np.random.randint(0, x.size(-1) - int(cutmix_max * x.size(-1)))
    bbx_y2 = bbx_y + int(cutmix_max * x.size(-2))
    bbx_x2 = bbx_x + int(cutmix_max * x.size(-1))
    mask[indices, :, bbx_y:bbx_y2, bbx_x:bbx_x2] = 0

    mixed_x = lambda_ * x * mask + (1 - lambda_) * x[indices] * (1 - mask)

    # Mixed label using weighted average
    mixed_y = lambda_ * y + (1 - lambda_) * y[indices]

    return mixed_x, mixed_y


# Define custom loss function and KL divergence
def KL(alpha, num_classes=10):
    one = torch.ones((1, num_classes), dtype=torch.float32)
    S = torch.sum(alpha, dim=1, keepdim=True)

    kl = torch.lgamma(S) - torch.sum(torch.lgamma(alpha), dim=1, keepdim=True) + \
         torch.sum(torch.lgamma(one), dim=1, keepdim=True) - torch.lgamma(torch.sum(one, dim=1, keepdim=True)) + \
         torch.sum((alpha - one) * (torch.digamma(alpha) - torch.digamma(S)), dim=1, keepdim=True)

    return kl

def custom_loss(y_true, output):
    epochs = [1]

    y_evidence = torch.relu(output)
    alpha = y_evidence + 1
    S = torch.sum(alpha, dim=1, keepdim=True)
    p = alpha / S

    err = torch.sum(torch.pow((y_true - p), 2), dim=1, keepdim=True)
    var = torch.sum(alpha * (S - alpha) / (S * S * (S + 1)), dim=1, keepdim=True)

    l = torch.sum(err + var, dim=1, keepdim=True)

    kl = torch.min(torch.tensor(1.0), torch.tensor(epochs[0] / 50)) * torch.sum(KL((1 - y_true) * (alpha) + y_true))
    return torch.sum(l + kl)

# Train LeNet model with Dirichlet distribution and regmixup for Evidential Deep Learning
lenet_dirichlet_CIFAR_mixup = LeNetDirichletCIFARMixup()
optimizer_dirichlet = optim.Adam(lenet_dirichlet_CIFAR_mixup.parameters(), lr=0.001)

for epoch in range(10):
    lenet_dirichlet_CIFAR_mixup.train()
    for inputs, labels in train_loader:
        inputs, labels = regmixup(inputs, labels.unsqueeze(1))  # Assuming labels are one-dimensional

        optimizer_dirichlet.zero_grad()
        outputs, _, _ = lenet_dirichlet_CIFAR_mixup(inputs)

        # Cast labels back to Long data type
        labels = labels.squeeze().long()

        loss = custom_loss(F.one_hot(labels, num_classes=10).float(), outputs)  # Using custom loss
        loss.backward()
        optimizer_dirichlet.step()

    lenet_dirichlet_CIFAR_mixup.eval()
    correct_train = 0
    total_train = 0
    with torch.no_grad():
        for inputs, labels in train_loader:
            outputs, uncertainty_train , _ = lenet_dirichlet_CIFAR_mixup(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs, uncertainty_test , _ = lenet_dirichlet_CIFAR_mixup(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()

    print('epoch %d - training accuracy: %2.4f \t training uncertainty: %2.4f \t testing accuracy: %2.4f \t testing uncertainty: %2.4f' %
          (epoch+1, correct_train / total_train, uncertainty_train.mean(), correct_test / total_test, uncertainty_test.mean()))

100%|██████████| 170498071/170498071 [00:04<00:00, 40890326.66it/s]


Extracting ./data1/cifar-10-python.tar.gz to ./data1
Files already downloaded and verified
epoch 1 - training accuracy: 0.4344 	 training uncertainty: 0.7060 	 testing accuracy: 0.4387 	 testing uncertainty: 0.6772
epoch 2 - training accuracy: 0.5121 	 training uncertainty: 0.7256 	 testing accuracy: 0.5045 	 testing uncertainty: 0.6903
epoch 3 - training accuracy: 0.5276 	 training uncertainty: 0.6137 	 testing accuracy: 0.5162 	 testing uncertainty: 0.5216
epoch 4 - training accuracy: 0.5835 	 training uncertainty: 0.7073 	 testing accuracy: 0.5651 	 testing uncertainty: 0.5575
epoch 5 - training accuracy: 0.5946 	 training uncertainty: 0.5850 	 testing accuracy: 0.5706 	 testing uncertainty: 0.4733
epoch 6 - training accuracy: 0.6307 	 training uncertainty: 0.4513 	 testing accuracy: 0.6012 	 testing uncertainty: 0.4369
epoch 7 - training accuracy: 0.6276 	 training uncertainty: 0.5109 	 testing accuracy: 0.5950 	 testing uncertainty: 0.4224
epoch 8 - training accuracy: 0.6404 	 tra